In [176]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_bounds
from sentinelhub import (
    SHConfig,
    SentinelHubRequest,
    DataCollection,
    MimeType,
    CRS,
    BBox,
    bbox_to_dimensions
)
from dotenv import load_dotenv
from shapely.geometry import box
import geopandas as gpd

We want to extract all the city of Quito in a grid of images that cover the entire city, we this approach we hope
- **Complete spatial coverage**
- **Structured data for deep learning**: So the model can learn patterns from individual tiles while mantaining an organized spatial representation.
- **Comparison  across areas**
- **Scalability**

To define the grid size is important to have the best detail as possible, Sentinel-2 has 10m, 20m and 60m resolutions, we aim to use 10m pixels, and a grid of 1 $km^{2}$, we need to ensure constent images, beware of clouds 

In [177]:
# Load SentinelHub API credentials
load_dotenv()

True

In [187]:
config = SHConfig()
config.sh_client_id = os.getenv("COPERNICUS_CLIENT_ID")
config.sh_client_secret = os.getenv("COPERNICUS_CLIENT_SECRET")
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

In [188]:
# Define Quito's bounding box (approximate)
quito_bbox_wgs84 = [-78.6, -0.4, -78.3, -0.1]  # [min_lon, min_lat, max_lon, max_lat]
resolution = 10  # Sentinel-2 resolution (10m per pixel)
tile_size_km = 1  # Grid tile size (1km²)
tile_size_meters = tile_size_km * 1000  # Convert km to meters

In [189]:
# Generate grid over Quito
def generate_grid(bbox, tile_size):
    """Generates a grid of bounding boxes within a larger bounding box."""
    minx, miny, maxx, maxy = bbox
    grid_tiles = []
    for x in np.arange(minx, maxx, tile_size / 111320):  # Convert meters to degrees
        for y in np.arange(miny, maxy, tile_size / 111320):
            grid_tiles.append(BBox(bbox=[x, y, x + (tile_size / 111320), y + (tile_size / 111320)], crs=CRS.WGS84))
    return grid_tiles

grid_boxes = generate_grid(quito_bbox_wgs84, tile_size_meters)

print(f"Generated {len(grid_boxes)} tiles over Quito.")

Generated 1156 tiles over Quito.


In [190]:
# Fetch images for each grid tile
output_dir = "quito_satellite_images"
os.makedirs(output_dir, exist_ok=True)

output_dir_uint16 = "quito_satellite_images/uint16"
output_dir_uint8 = "quito_satellite_images/uint8"
os.makedirs(output_dir_uint16, exist_ok=True)
os.makedirs(output_dir_uint8, exist_ok=True)

In [191]:
# Define Evalscript for Multi-Band Data (True Color + NIR + SWIR) NIR and SWIR bands useful to detect urban infraestructure, water access, vegetation and green
evalscript_multiband = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "B08", "B11", "B12", "SCL"]
            }],
            output: {
                bands: 6,
                sampleType: "UINT16"
            }
        };
    }

    function evaluatePixel(sample) {
        // Sentinel-2 Scene Classification Layer (SCL) values:
        // 3 = Vegetation, 4 = Bare Soil, 5 = Water, 6 = Urban/Artificial
        // 9 = Cirrus Clouds, 10 = High Clouds, 11 = Thin Clouds
        if (sample.SCL == 3 || sample.SCL == 4 || sample.SCL == 5 || sample.SCL == 6) {
            return [
                sample.B02 * 10000,  // Blue
                sample.B03 * 10000,  // Green
                sample.B04 * 10000,  // Red
                sample.B08 * 10000,  // Near-Infrared (NIR)
                sample.B11 * 10000,  // SWIR1
                sample.B12 * 10000   // SWIR2
            ];
        } else {
            return [0, 0, 0, 0, 0, 0];  // Mask clouds as black
        }
    }

"""

In [192]:
# Define Evalscript for true color just RGB bands for human visualization
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "SCL"]
            }],
            output: {
                bands: 3,
                sampleType: "UINT8"
            }
        };
    }

    function evaluatePixel(sample) {
        // Remove clouds using SCL (scene classification)
        if (sample.SCL == 3 || sample.SCL == 4 || sample.SCL == 5 || sample.SCL == 6) {
            return [
                sample.B04 * 255,  // Red
                sample.B03 * 255,  // Green
                sample.B02 * 255   // Blue
            ];
        } else {
            return [0, 0, 0];  // Mask clouds as black
        }
    }
"""

In [193]:
for idx, tile_bbox in enumerate(grid_boxes):
    print(f"Processing tile {idx+1}/{len(grid_boxes)}...")

    tile_size = bbox_to_dimensions(tile_bbox, resolution=resolution)

    # Request for Multi-Band TIFF (UINT16)
    request_tiff = SentinelHubRequest(
        evalscript=evalscript_multiband,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
                ),
                time_interval=("2022-05-01", "2022-05-20"),
                other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=tile_bbox,
        size=tile_size,
        config=config,
    )

    tile_img_tiff = request_tiff.get_data()[0]

    # Save Multi-Band Image (UINT16 TIFF)
    tile_tiff_path = os.path.join(output_dir_uint16, f"tile_{idx}.tiff")

    # Adding georefence
    minx, miny, maxx, maxy = tile_bbox  # Get coordinates from SentinelHub BBox
    transform = from_bounds(minx, miny, maxx, maxy, tile_img_tiff.shape[1], tile_img_tiff.shape[0])

    with rasterio.open(
        tile_tiff_path,
        'w',
        driver='GTiff',
        height=tile_img_tiff.shape[0],
        width=tile_img_tiff.shape[1],
        count=tile_img_tiff.shape[2],  # Number of bands
        dtype=rasterio.uint16,
        crs='EPSG:4326', # Assign correct projection (WGS84)
        transform=transform,  # Add geotransform if available
    ) as dst:
        for band in range(tile_img_tiff.shape[2]):
            dst.write(tile_img_tiff[:, :, band], band + 1)

    print(f"{idx} Saved Multi-Band GeoTIFF: {tile_tiff_path}")

    # Request for True Color PNG (UINT8)
    request_png = SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
                ),
                time_interval=("2022-05-01", "2022-05-20"),
                other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=tile_bbox,
        size=tile_size,
        config=config,
    )

    tile_img_png = request_png.get_data()[0]

    # Save RGB Image as PNG for Visualization
    tile_png_path = os.path.join(output_dir_uint8, f"tile_{idx}.png")
    plt.imsave(tile_png_path, tile_img_png)

    print(f"{idx} Saved True Color PNG: {tile_png_path}")

print("All tiles processed and saved successfully!")

Processing tile 1/1156...
0 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_0.tiff
0 Saved True Color PNG: quito_satellite_images/uint8/tile_0.png
Processing tile 2/1156...
1 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_1.tiff
1 Saved True Color PNG: quito_satellite_images/uint8/tile_1.png
Processing tile 3/1156...
2 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_2.tiff
2 Saved True Color PNG: quito_satellite_images/uint8/tile_2.png
Processing tile 4/1156...
3 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_3.tiff
3 Saved True Color PNG: quito_satellite_images/uint8/tile_3.png
Processing tile 5/1156...
4 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_4.tiff
4 Saved True Color PNG: quito_satellite_images/uint8/tile_4.png
Processing tile 6/1156...
5 Saved Multi-Band GeoTIFF: quito_satellite_images/uint16/tile_5.tiff
5 Saved True Color PNG: quito_satellite_images/uint8/tile_5.png
Processing tile 7/1156...
6 Saved Multi-